In [ ]:
# First we import all the libraries that we are going to require.....

In [1]:
import pandas as pd
import h5py
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

In [33]:
# We read the Photons datasets. ......we denote them by 0

In [2]:
with h5py.File("D:\\Datasets\\GSoc\\Photons\\SinglePhotonPt50_IMGCROPS_n249k_RHv1.hdf5", mode='r') as hdf:
    ls = list(hdf.keys())
    photons = np.array(hdf.get('X'))


In [ ]:
# We read the electrons dataset .... we denote them by 1

In [3]:
with h5py.File('D:\\Datasets\\GSoc\\Electrons\\SingleElectronPt50_IMGCROPS_n249k_RHv1.hdf5', mode='r') as hdf:
    ls = list(hdf.keys())
    electrons = np.array(hdf.get('X'))
    

In [4]:
electrons[0].shape, photons.shape

((32, 32, 2), (249000, 32, 32, 2))

In [ ]:
# Creating the dataset

In [5]:
class Data(Dataset):
    def __init__(self, transform):
        super(Data, self).__init__()
        self.transform = transform
        self.data = []
        for val in electrons:
            val = self.transform(val)
            self.data.append((val, 1))

        for val in photons:
            val = self.transform(val)
            self.data.append((val, 0))

    def __getitem__(self, item):
        return torch.Tensor(self.data[item][0]), torch.Tensor(self.data[item][1])

    def __len__(self):
        return (electrons.shape[0] + photons.shape[0])


In [6]:
transform = transforms.Compose([
    transforms.ToTensor()
])

In [7]:
all_data = Data(transform)

In [19]:
# Split the data into train and test

In [9]:
from torch.utils.data import random_split

In [10]:
train_size = int(len(all_data) * 0.9)
test_size = len(all_data) - train_size


In [11]:
train_data, test_data = random_split(all_data, [train_size, test_size])

In [12]:
train_batch = DataLoader(
    train_data,
    batch_size=128,
    shuffle=True,
    num_workers=8
)

In [13]:
test_batch = DataLoader(
    test_data,
    batch_size=128,
    shuffle=True,
    num_workers=8
)

In [ ]:
for (img, label) in train_batch:
    print(img.shape, label.shape)


In [ ]:
# create the model

In [14]:
class Block(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=(1, 1), padding=(0, 0)):
        super(Block, self).__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )

    def forward(self, x):
        return self.block(x)

In [15]:
class MyModel(nn.Module):
    def __init__(self, in_channels, layers=[32, 64, 128, 256, 512]):
        super(MyModel, self).__init__()
        self.model = nn.Sequential(
            Block(in_channels, layers[0], (3, 3)),
            Block(layers[0], layers[1], (3, 3), (2, 2)),
            Block(layers[1], layers[2], (3, 3), (2, 2), (1, 1)),
            Block(layers[2], layers[3], (3, 3), padding=(1, 1)),
            nn.Dropout2d(0.2),
            nn.MaxPool2d((2, 2), (2, 2)),
            Block(layers[3], layers[4], (1, 1)),
            nn.Flatten(),
            nn.Linear(4608, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 2)
        )

    def forward(self, x):
        return self.model(x)


In [ ]:
# loss function

In [16]:
loss = torch.nn.CrossEntropyLoss()

In [ ]:
# Optimizer

In [17]:
model = MyModel(in_channels=2)
opti = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Hyperparameters

In [18]:
epochs = 3
lr = 0.001

In [ ]:
# To check accuracy

In [19]:
def calculate_accuracy(y_hat, y):
    tot = 0
    correct = 0
    
    for i, val in y:
        pred = torch.argmax(y_hat[i])
        if pred == val:
            correct += 1
        tot += 1
        
    return (correct / tot)

In [ ]:
# Testing Function

In [21]:
def test_fun(model, batch):
    correct = 0
    tot = 0
    
    with torch.no_grad():
        for (img, label) in batch:
            out = model(img)
        
            for i, val in label:
                if torch.argmax(out[i]) == val:
                    correct += 1
                
                tot += 1
    return (correct/ tot)

In [ ]:
# Training

In [ ]:
for epoch in range(epochs):
    for (img, labels) in train_batch:
        out = model(img)
        loss_val = loss(out, labels)
        print(f"for epoch {epoch} loss is {loss} accuracy is {calculate_accuracy(out, labels)}")
        loss_val.backward()
        opti.step()
        opti.zero_grad()

print(f"accuracy on test set is {test_fun(model,  test_batch)}")